# Exercise 11 - Pass Neural Net Weights Between Processes

**GOAL:** The goal of this exercise is to show how to send neural network weights between workers and the driver.

For more details on using Ray with TensorFlow, see the documentation at http://ray.readthedocs.io/en/latest/using-ray-with-tensorflow.html.

### Concepts for this Exercise - Getting and Setting Neural Net Weights

Since pickling and unpickling a TensorFlow graph can be inefficient or may not work at all, it is most efficient to ship the weights between processes as a dictionary of numpy arrays (or as a flattened numpy array).

We provide the helper class `ray.experimental.TensorFlowVariables` to help with getting and setting weights. Similar techniques should work other neural net libraries.

Consider the following neural net definition.

```python
import tensorflow as tf

x_data = tf.placeholder(tf.float32, shape=[100])
y_data = tf.placeholder(tf.float32, shape=[100])

w = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))
y = w * x_data + b

loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
grads = optimizer.compute_gradients(loss)
train = optimizer.apply_gradients(grads)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
```

Then we can use the helper class as follows.

```python
variables = ray.experimental.TensorFlowVariables(loss, sess)
# Here 'weights' is a dictionary mapping variable names to the associated
# weights as a numpy array.
weights = variables.get_weights()
variables.set_weights(weights)
```

Note that there are analogous methods `variables.get_flat` and `variables.set_flat`, which concatenate the weights as a single array instead of a dictionary.

```python
# Here 'weights' is a numpy array of all of the neural net weights
# concatenated together.
weights = variables.get_flat()
variables.set_flat(weights)
```

In this exercise, we will use an actor containing a neural network and implement methods to extract and set the neural net weights.

**WARNING:** This exercise is more complex than previous exercises.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import ray
import tensorflow as tf
import time

In [ ]:
ray.init(num_cpus=4, include_webui=False, ignore_reinit_error=True)

The code below defines a class containing a simple neural network.

**EXERCISE:** Implement the `set_weights` and `get_weights` methods. This should be done using the `ray.experimental.TensorFlowVariables` helper class.

In [ ]:
@ray.remote
class SimpleModel(object):
    def __init__(self):
        x_data = tf.placeholder(tf.float32, shape=[100])
        y_data = tf.placeholder(tf.float32, shape=[100])

        w = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
        b = tf.Variable(tf.zeros([1]))
        y = w * x_data + b

        self.loss = tf.reduce_mean(tf.square(y - y_data))
        optimizer = tf.train.GradientDescentOptimizer(0.5)
        grads = optimizer.compute_gradients(self.loss)
        self.train = optimizer.apply_gradients(grads)

        init = tf.global_variables_initializer()
        self.sess = tf.Session()

        # Here we create the TensorFlowVariables object to assist with getting
        # and setting weights.
        self.variables = ray.experimental.TensorFlowVariables(self.loss, self.sess)

        self.sess.run(init)

    def set_weights(self, weights):
        """Set the neural net weights.
        
        This method should assign the given weights to the neural net.
        
        Args:
            weights: Either a dict mapping strings (the variable names) to numpy
                arrays or a single flattened numpy array containing all of the
                concatenated weights.
        """
        # EXERCISE: You will want to use self.variables here.
        raise NotImplementedError

    def get_weights(self):
        """Get the neural net weights.
        
        This method should return the current neural net weights.
        
        Returns:
            Either a dict mapping strings (the variable names) to numpy arrays or
                a single flattened numpy array containing all of the concatenated
                weights.
        """
        # EXERCISE: You will want to use self.variables here.
        raise NotImplementedError

Create a few actors.

In [ ]:
actors = [SimpleModel.remote() for _ in range(4)]

**EXERCISE:** Get the neural net weights from all of the actors.

In [ ]:
raise Exception('Implement this.')

**EXERCISE:** Average all of the neural net weights.

**NOTE:** This will be easier to do if you chose to use `get_flat`/`set_flat` instead of `get_weights`/`set_weights` in the implementation of `SimpleModel.set_weights` and `SimpleModel.get_weights` above..

In [ ]:
raise Exception('Implement this.')

**EXERCISE:** Set the average weights on the actors.

In [ ]:
raise Exception('Implement this.')

**VERIFY:** Check that all of the actors have the same weights.

In [ ]:
weights = ray.get([actor.get_weights.remote() for actor in actors])

for i in range(len(weights)):
    np.testing.assert_equal(weights[i], weights[0])

print('Success! The test passed.')